In this file, you can find the source code for the Homework 5 of the Cryptography and Security course. It consists of **4 exercises**. All of the material here are my own work. In case of doubt, please send an e-mail to: [furkan.karakas@epfl.ch](mailto:furkan.karakas@epfl.ch)

# Exercise 1

In [129]:
Q1_p = 60055542378743800870984650064300434749863701922850044406431317093041369825985827433749546393895540449652025734272324325766218491281694238130722173393183483885437811760557394111938168661132034686043294099496516186034464224769515668824134222379411213622533288466031472830765575070050471211910512309277394625511039608604466794511486442978890755447340853298788925954132849494246405952766545954686681263626373302433301707202080315167178326221865198357311916159655281789965111665708959524382410794912853864178879967461309707005917351897844330144906695159023225938171656971885521551778839866449272779226789329
Q1_q = 946369383149503913097499114667945567529167788584664587545246699802207806093
Q1_g = 22493437217519503842042074486862187182867236296232324847042667312311970684978888420271944247698412501149798133622445424915931079950752980864405692539831020246953893352433357520362036811397373869313049949339622279744263417333262727784136625059611007301281594242808226739999769715591223297216365080543802674283076625008089439983472611892820555332588466330669644978677276389400870722474404883111442691947196879407967650259989043439266848164547113521977979571296250750760221388613710711549516681789079332853324508231151989505119703100522561290674805773702857445297920694801675555969359295881697509797401703
Q1_pk = 21579122373810148192426679570254459566706893802381487686437604654447040236999389394976588078137350765863998913497352097856241487709489742471867379385292101222933058373422794866703600382017516146075680143923425104545511469128173165960864663716998960694450620128147832644049946062302864233745734462128043444264712363779142880385211859214003064816146726283277154126863676662362503056240493674605657844936879823395157933374140748012953381882814219445614375342985337516927906951670159733807663593315180226758818276264267474673360532223388772090646453332583892693036531300377909449546273767022021563693255423
Q1_sk = 904377269404065580142442670307799592768005931814952873288008284838402250185

In [130]:
import socket

def connect_server(server_name, port, message):
    server = (server_name, int(port)) #calling int is required when using Sage
    s = socket.create_connection(server)
    s.send(bytes(message + "\n"))
    response=""
    while True: #data might come in several packets, need to wait for all of it
        data = s.recv(1024)
        data = data.decode("ASCII")
        if len(data) == 0:
            break
        if data[-1] == '\n': 
            response += data[:-1]  
            break
        response += data
    s.close()
    return response


def oracle_query(sciper):
    server_name = "lasecpc25.epfl.ch"
    port = "5559"
    message = sciper 
    response = connect_server(server_name, port, message)
    #print(response)
    return response


In [131]:
print floor(log(Q1_p,2))+1

2000


In [132]:
response = oracle_query('306399')

In [133]:
len_p = floor(log(Q1_p,2))+1

In [134]:
parameter_l = (len(response) - 256/4 - 3*2000/4) / 64
print parameter_l

41


In [135]:
import hashlib
import binascii

def XOR(a,b):
    assert len(a) == len(b)
    result=''
    for i in xrange(len(a)):
        if a[i]==b[i]:
            result+='0'
        else:
            result+='1'
    return result

def convert_hex_to_binary(k_0):
    k_0_len=len(k_0)
    k_0=int(k_0,16) # k_0 in integer form
    k_0=bin(k_0) # k_0 in binary string
    k_0=k_0[2:] # get rid of '0b' at the beginning
    k_0='0'*(k_0_len*4 - len(k_0)) + k_0 # Add some zeros at the start if we lost some along the way
    return k_0

def convert_binary_to_ascii(input_to_sha256):
    input_to_sha256=int(input_to_sha256,2)
    input_to_sha256=hex(input_to_sha256)
    # Get rid of the annoying L at the end of the string
    if input_to_sha256[-1] == 'L':
        input_to_sha256=input_to_sha256[0:-1]
    # Get rid of the first '0x' string
    if input_to_sha256[0:2] == '0x':
        input_to_sha256=input_to_sha256[2:]
    assert len(potential_St)%4 == 0
    input_to_sha256='0'*(len(potential_St)//4 - len(input_to_sha256)) + input_to_sha256 # Add some zeros in case they got removed during conversion
    input_to_sha256=binascii.unhexlify(input_to_sha256) # Convert it to ASCII string
    return input_to_sha256

def is_ascii(s):
    return all(ord(c) < 128 for c in s)

def KDF(potential_St):
    input_to_sha256=int(potential_St,2)
    input_to_sha256=hex(input_to_sha256)
    # Get rid of the annoying L at the end of the string
    if input_to_sha256[-1] == 'L':
        input_to_sha256=input_to_sha256[0:-1]
    # Get rid of the first '0x' string
    if input_to_sha256[0:2] == '0x':
        input_to_sha256=input_to_sha256[2:]
    assert len(potential_St)%4 == 0
    input_to_sha256='0'*(len(potential_St)//4 - len(input_to_sha256)) + input_to_sha256 # Add some zeros in case they got removed during conversion
    input_to_sha256=input_to_sha256.decode('hex') # Decode it to bytes
    k_0=hashlib.sha256('\x00'+input_to_sha256).hexdigest() # This is k_0, in hexadecimal string
    k_0=convert_hex_to_binary(k_0) # This is the final k in binary string form
    # Now, calculate St'
    St_prime=hashlib.sha256('\xff'+input_to_sha256).hexdigest() # This is St', in hexadecimal string
    St_prime=convert_hex_to_binary(St_prime) # This is the final St' in binary string form
    return (St_prime,k_0)

In [139]:
while True:
    response = oracle_query('306399')

    response=str(response)

    group_membership=[]
    for i in xrange(len(response)-500):
        candidate=response[i:i+500]
        temp=int(candidate,16)
        if power_mod(temp,Q1_q,Q1_p) == 1:
            group_membership.append(candidate)

    find_positions=[response.find(x) for x in group_membership]
    if len(find_positions) != 3:
        continue
    
    messages_and_hash=response[0:find_positions[0]]+response[find_positions[0]+500:find_positions[1]]+response[find_positions[1]+500:find_positions[2]]+response[find_positions[2]+500:]
    assert len(messages_and_hash)%64 == 0
    messages_and_hash=[messages_and_hash[64*i:64*(i+1)] for i in xrange(len(messages_and_hash)//64)]
    assert len(messages_and_hash) == parameter_l+1 # 1 for the tag

    parameter_t=-1
    parameter_X0=''
    message=''
    for potential_X0 in group_membership:
        tempvar=potential_X0
        potential_X0=int(potential_X0,16)
        potential_St=power_mod(potential_X0,Q1_sk,Q1_p)
        potential_St=bin(potential_St)
        assert len(potential_St) >= 258 # Including first '0b', assert that the length is indeed greater than or equal to 258
        potential_St=potential_St[-256:] # This is the last 256 bits in string format
        for i in xrange(parameter_l): # Let's gamble
            (potential_St,k_0)=KDF(potential_St)
            for piece_of_message in messages_and_hash: # 64 because we have 32 characters -> 32*8 bits -> 32*8/4=64 hexedecimal characters in a piece of message
                piece_of_message=convert_hex_to_binary(piece_of_message)
                potential_message=XOR(piece_of_message,k_0)
                potential_message=convert_binary_to_ascii(potential_message)
                # print len(potential_message)
                if is_ascii(potential_message):
                    message+=potential_message
                    parameter_t+=1
                    parameter_X0=tempvar
                    break
    if 'password' in message:
        print message
        break

Host: www.facebook.com
User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:70.0) Gecko/20100101 Firefox/70.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate, br
Referer: https://www.facebook.com/
Content-Type: application/x-www-form-urlencoded
Content-Length: 717
Origin: https://www.facebook.com
Connection: keep-alive
Cookie: datr=OS3gXbc0qOW7dUCgL8qLbxfS; fr=1RwNHd0bQbOEtQD3s..Bd4C0_.vY.AAA.0.0.Bd4C0_.AWWvJ2NW; sb=Py3gXSmeUrcJjql7pX6-Ay7r; wd=1618x441
Upgrade-Insecure-Requests: 1
TE: Trailers
jazoest=2664&lsd=AVoh5v9F&email=apprentice@facebook.com&password=ylX-VGg-KR6-nJo&timezone=-60&lgndim=eyJ3IjoxOTIwLCJoIjoxMDgwLCJhdyI6MTg2NCwiYWgiOjEwODAsImMiOjI0fQ%3D%3D&lgnrnd=122536_zNE6&lgnjs=1574972737&ab_test_data=AAAf%2FfA%2FAfAAfAAAAAAAAAAAAAAAAAAAAAAAAAAAb%2F%2FbSSAAJAEGAB&locale=en_US&next=https%3A%2F%2Fwww.facebook.com%2F&login_source=login_bluebar&guid=fa0d9ade4d308c&prefill_contact_point=&prefill

In [137]:
temp1,temp2=KDF('0011000100110001001100010011000100110001001100010011000100110001001100010011000100110001001100010011000100110001001100010011000100110001001100010011000100110001001100010011000100110001001100010011000100110001001100010011000100110001001100010011000100110001')
print hex(int(temp1,2)),'********',hex(int(temp2,2))
print hashlib.sha256('\xff11111111111111111111111111111111').hexdigest(),'********',hashlib.sha256('\x0011111111111111111111111111111111').hexdigest()
print hex(int(temp1,2))[2:-1] == hashlib.sha256('\xff11111111111111111111111111111111').hexdigest()
print hex(int(temp2,2))[2:-1] == hashlib.sha256('\x0011111111111111111111111111111111').hexdigest()

0xbde053c15aca51e9f3012f2064ffcbeea346e0ec685974f4f3375b94dde1db0aL ******** 0xf57ce4b9e48271abcc01f76a20f34fdffb87b34b11ba81bc77f77dcfd5880ab9L
bde053c15aca51e9f3012f2064ffcbeea346e0ec685974f4f3375b94dde1db0a ******** f57ce4b9e48271abcc01f76a20f34fdffb87b34b11ba81bc77f77dcfd5880ab9
True
True


# Exercise 2

In [83]:
Q2_r = 352
Q2_c = 160
Q2_d = 704
Q2_m = "Tr50yTuIDk/WhVrYxfPTOEyHfrhPchyE5Aeq7KEeB8b+Q1aiL7h1aO6SgeTLDnimxALo8VAyauFj3ylmn2LgtuGGKLn8hasZjFbe6pF9pHOBFZGpR1VE2OeLzIyrgWeIemE8lnYyml0r83Fz/beH59hZPx/Nn0ouY3qdTEy5w331TBrkIsOm/w2IYvgp8d0GtPfTLaGXmrn/mKfpsf/iHv6opGZ3pxMZrwd2h34xI97pccfYheqJ4z3C0KPWPXiN9YSI23EJwO7A84Dd8omf/tRluRSERvVgFFbmoghX/e+pEaLQoNPVaNhq3boLVA7I1PrGCa+lKzC5Mk4VUGWB1a2peii1QYzI8edH7uxVLJ2Op5aZmQo/u2JgxA7Ikx9xT4yUZDNR8mu5Wn8SZf8Ya8bfrfGMoqXi1FiTt+yjwpit4stlEit0qhEoJ1dpOvRZv+fqUoJMQLDhlU7cKZor2id7PMhaJubQFeUFel/r6z8FY5SYf/9k5Pg8GPta5Z5ZqiSFUano+u2EKeUX2VTTdGY89U7x9anqGIrFrm0A8liqz9+KJMUzgl6CyO+qzzFPUh5JSAMUiW8Vu2pFRuUukCFUTFYuTdMvBjkRiUAMMEZcqg2o09zgd1D7LduG9yjFAR7K5ccs7qTLQsqbRO2pMkaUGro7Dm8C15tpb0BHLidTv4i6CRBimcY1JJzwhdqkp5jrsraMkjL4EnJXGJonfIKtmeuCRj2Uol54NlZPQN+mgQSWBIw5UcC2xmQBiE6WzM7ohbALHu1fQOqbgDhyPvdZtUdRY+kp9/QzHpFmgEPtNsf1UV832SZrcdrbSmvoWmpBlmUOj7RC9cYbm3Rrx+dmGvhOJGG416H6mD/59Obs4UkdyNmp+YP4crYAOCL+JVqTP0D5f/C2r1riGnA8wQeUZaAgQVIhrt2vJX1IJdUZicnuBhzD5EVD8PPpe/s9ObAdtceqH37BBmmjixIehDWUZzDPDWfPn7zqrbSofa9NjgEGCFIb9Mj0e2nZQn9hjkyuY0kvnQbM6r//hKvmSLzs7Y4Ac6B0fZPrK3VvgtNdyUhpqCwRlrVO107nHbBatVwVvG4gwJPUlYBz5CUkEZ3DA3er7fdVY9doscQQ8WUhFfKTf9VYO/zpbtQhvlCuACWSVCuzDJ4YP2ts4wLazzFfqi/h/0M62SSBdU/3QuXV1NzJlqPT36Sa+nJZVbmN984ilNOlmATuw1vWUKhxW+n5RcrpuSZydcuniseyNmDf1N2GL3ElbUv7GRbfM2/sr1XjxtlqBWEE2/AGzOpdRsJiiayOMTgl+E7M9jE7xB6JZyJSxQr7/swLAU1JXCdCVU6paFWwaUUbRZUGGKrcq5j6vpV2bLI1B08HSA9gwK3zkcBlxUTb6CmBMoyGIILgvG8B7Tg5e0xzECAVOo7X3QABxnKUb+08hGLhz+SLAJUD31ZWUsYu8nG54Ms8heMvfJe2HOnlZUS03np4oCCdyiGi7Pg5y2hN8NpR9WfedS6ivvo4RZrSqtU3"

In [84]:
import hashlib
import base64

def XOR(a,b):
    assert len(a) == len(b)
    result=''
    for i in xrange(len(a)):
        if a[i]==b[i]:
            result+='0'
        else:
            result+='1'
    return result

def convert_to_hex(x):
    assert len(x)%8 == 0
    x_len = len(x)
    assert x_len%4 == 0
    temp=int(x,2)
    temp=hex(temp)
    if temp[0:2] == '0x':
        temp=temp[2:]
    if temp[-1] == 'L':
        temp=temp[0:-1]
    temp=(x_len/4 - len(temp))*'0' + temp
    try:
        temp=temp.decode('hex')
    except:
        temp='0'+temp
        print type(temp)
        temp=temp.decode('hex')
    return temp

def base64_decoder(m):
    temp=base64.decodestring(m).encode('hex')
    lentemp = len(temp)
    temp=int(temp,16)
    temp=bin(temp)
    temp=temp[2:]
    temp='0'*(lentemp*4-len(temp))+temp
    return temp

def base64_encoder(x):
    x_len = len(x)
    assert x_len%4 == 0
    temp=int(x,2)
    temp=hex(temp)
    if temp[0:2] == '0x':
        temp=temp[2:]
    if temp[-1] == 'L':
        temp=temp[0:-1]
    temp=(x_len/4 - len(temp))*'0' + temp
    try:
        temp=temp.decode('hex')
    except:
        temp='0'+temp
        print temp
        temp=temp.decode('hex')
    temp=base64.encodestring(temp)
    temp=temp.replace('\n','')
    return temp

In [85]:
m=base64_decoder(Q2_m)
m = m + '10' + str( 10^( -(len(m)+4) % Q2_r ) ) + '1'
assert len(m)%Q2_r == 0
m_array = [m[Q2_r*i:Q2_r*(i+1)] for i in xrange(len(m)/Q2_r)]
s='0'*(Q2_r+Q2_c)
for i in m_array:
    temp=bin(int(hashlib.sha512(convert_to_hex(XOR(s,i+('0'*Q2_c)))).hexdigest(),16))[2:]
    s='0'*(512-len(temp))+temp
h=''
for i in xrange(floor(Q2_d/Q2_r)):
    h+=s[0:Q2_r]
    temp=bin(int(hashlib.sha512(convert_to_hex(s)).hexdigest(),16))[2:]
    s='0'*(512-len(temp))+temp

if Q2_d%Q2_r != 0:
    h+=s[0:(Q2_d%Q2_r)]
res=base64_encoder(h)

print repr(res)

'mtrY7K3oUOTLONNLybGXgHvm1xjRmw/GsmgYPtnFzHJdRBn+9Q8rKHIDOb50xuhnRXI/AaxEl6We/fJ7qZKBAmAsmUcKUqQdU2uIGJsZJL520T5hQlTD4g=='


In [86]:
# This will evaluate to false in my personal parameters
print 'kKJn3mR4kHm3paeHT1LxkNfo0y0WtJuL8oaMJNHtN10H9iEsWNv/1x1XyzRgPJaTRW2n9nSA44lrslfZlcVOOQonj0SVtXVrn045cuFefBvY4Kg1' == res

False


# Exercise 3

In [20]:
###### Exercise 3
Q3_id = "wwzifYrHPFpSJn"

In [21]:
import hashlib
import base64

def XOR(a,b):
    assert len(a) == len(b)
    result=''
    for i in xrange(len(a)):
        if a[i]==b[i]:
            result+='0'
        else:
            result+='1'
    return result

def convert_binary_to_hexbytes(x):
    assert len(x)%8 == 0
    x_len = len(x)
    assert x_len%4 == 0
    temp=int(x,2)
    temp=hex(temp)
    if temp[0:2] == '0x':
        temp=temp[2:]
    if temp[-1] == 'L':
        temp=temp[0:-1]
    temp=(x_len/4 - len(temp))*'0' + temp
    temp=temp.decode('hex')
    return temp

def base64_decode_to_binarystring(m):
    temp=base64.decodestring(m).encode('hex')
    lentemp = len(temp)
    temp=int(temp,16)
    temp=bin(temp)
    temp=temp[2:]
    temp='0'*(lentemp*4-len(temp))+temp
    return temp

def base64_encode_binarystring(x):
    x_len = len(x)
    assert x_len%4 == 0
    temp=int(x,2)
    temp=hex(temp)
    if temp[0:2] == '0x':
        temp=temp[2:]
    if temp[-1] == 'L':
        temp=temp[0:-1]
    temp=(x_len/4 - len(temp))*'0' + temp
    try:
        temp=temp.decode('hex')
    except:
        temp='0'+temp
        print temp
        temp=temp.decode('hex')
    temp=base64.encodestring(temp)
    temp=temp.replace('\n','')
    return temp

def convert_hex_to_binarystring(x):
    if x[0:2] == '0x':
        x=x[2:]
    if x[-1] == 'L':
        x=x[0:-1]
    temp=int(x,16)
    temp=bin(temp)
    if temp[0:2] == '0b':
        temp=temp[2:]
    if temp[-1] == 'L':
        temp=temp[0:-1]
    temp='0'*(len(x)*4 - len(temp)) + temp
    return temp

def trunc(x,n):
    return x[0:n]

def sha2_512(x):
    temp=convert_binary_to_hexbytes(x)
    temp=hashlib.sha512(temp).hexdigest()
    temp=convert_hex_to_binarystring(temp)
    return temp

def convert_string_to_binary(x):
    result=''
    for i in x:
        temp=bin(ord(i))
        temp=temp[2:]
        temp='0'*(8-len(temp)) + temp
        result+=temp
    return result

In [22]:
import socket

def connect_server(server_name, port, message,(prevs0,prevs1)):
    s0=prevs0
    s1=prevs1
    response=''
    while 'i' in s0:
        server = (server_name, int(port)) #calling int is required when using Sage
        s = socket.create_connection(server)

        # Send the ID to the server
        s.send(bytes(message + "\n"))
        # Receive m1 here
        response=receive_a_response(s)
        m1=response
        m1=base64_decode_to_binarystring(m1) # Convert m1 to the binary form

        m0='0'*128
        temp=base64_encode_binarystring(m0)
        # Send m0 to the server
        s.send(bytes(temp + '\n'))

        # Calculate c
        temp=convert_string_to_binary(message)+m0+m1
        c=sha2_512(temp)
        c=trunc(c,256)
        
        kprev=None
        prevchallenge=None
        
        for i in xrange(0,32):
            
            k=c[8*i:8*(i+1)]
            k=int(k,2)
            k=k%128
            
            # Receive the challenge
            response=receive_a_response(s)
            if kprev is None:
                if s1[k]==s0[k]=='i':
                    # Hope that a1[k] is equal to a0[k]
                    s.send(bytes(response + '\n'))
                else:
                    # Now, we know the answer
                    a0_k=s0[k]
                    a1_k=XOR(s1[k],m1[k])
                    if a0_k==a1_k:
                        try:
                            assert a0_k==a1_k==response
                        except:
                            print a0_k,a1_k,response
                            break
                        s.send(bytes(a0_k + '\n'))
                    else:
                        s.send(bytes(XOR(response,'1') + '\n'))
                    #print 'Im here I shouldnt receive abort'
            else:
                if response in ['0','1']:
                    
                    if s1[kprev]==s0[kprev]=='i':
                        # This means that we have guessed the previous value right
                        s0[kprev]=prevchallenge
                        s1[kprev]=XOR(m1[kprev],prevchallenge)
                    else:
                        # Don't change the previous values since they were already correct
                        pass
                    
                    # Guess the new value
                    if s1[k]==s0[k]=='i':
                        # Hope that a1[k] is equal to a0[k]
                        s.send(bytes(response + '\n'))
                    else:
                        # Now, we know the answer
                        a0_k=s0[k]
                        a1_k=XOR(s1[k],m1[k])
                        if a0_k==a1_k:
                            try:
                                assert a0_k==a1_k==response
                            except:
                                print a0_k,a1_k,response
                                break
                            s.send(bytes(a0_k + '\n'))
                        else:
                            s.send(bytes(XOR(response,'1') + '\n'))
                        #print 'Im here I shouldnt receive abort'
                else:
                    # Maybe we got an accept or abort (no luck)
                    break
            
            # Update kprev and prevchallenge
            kprev=k
            prevchallenge=response
        s.close()
    return (s0,s1)


def oracle_query(username,(x,y)):
    server_name = "lasecpc25.epfl.ch"
    port = "7777"
    message = username
    response = connect_server(server_name, port, message,(x,y))
    #print(response)
    return response


def receive_a_response(s):
    response=''
    while True: #data might come in several packets, need to wait for all of it
        data = s.recv(1024)
        data = data.decode("ASCII")
        data = str(data)
        if len(data) == 0:
            return str(response)
        if data[-1] == '\n': 
            response += data[:-1]
            return str(response)
        response += data

In [23]:
temp=oracle_query(Q3_id,(['i']*128,['i']*128))
print temp

(['1', '1', '1', '0', '0', '0', '0', '0', '1', '0', '0', '1', '1', '0', '1', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '0', '1', '0', '1', '1', '1', '0', '1', '0', '0', '1', '0', '1', '0', '1', '1', '1', '1', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '1', '0', '1', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '1', '0', '0', '0', '1', '1', '0', '0', '0', '1', '1', '1', '1', '1', '1', '1', '0', '0', '1', '1', '1', '0', '1', '0', '1', '0', '0', '0', '1', '0', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '1'], ['1', '0', '0', '1', '1', '1', '0', '1', '1', '0', '1', '1', '0', '0', '1', '1', '0', '0', '0', '0', '1', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1', '1', '0', '0', '1', '0', '1', '0', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '1', '0', '1', '0', '1', '0', '1', '1', '0', '0', '0', '1', '0', '1', '1', '

In [24]:
s0_final=''
s1_final=''
for i in temp[0]:
    s0_final+=i
for i in temp[1]:
    s1_final+=i
print s0_final
print '*******'
print s1_final

11100000100110100010011000101110100101011110001000100010111011010010100000101111000110001111111001110101000101111101011101101111
*******
10011101101100110000110011101111001010001000110000100100101010110001011010001000000100111110111011101101101101101001011000001110


In [25]:
s0_final_bytes=convert_binary_to_hexbytes(s0_final)
s1_final_bytes=convert_binary_to_hexbytes(s1_final)
from Crypto.Cipher import AES
j=''
for i in xrange(16):
    j+='\x00'
cipher0=AES.new(key=j)
plain0 =cipher0.decrypt(s0_final_bytes)
print plain0
j=''
for i in xrange(16):
    j+='\xff'
cipher1=AES.new(key=j)
plain1 =cipher1.decrypt(s1_final_bytes)
print plain1
print plain1==plain0

e2Z2<9QB)d!*WZ  
e2Z2<9QB)d!*WZ  
True


In [26]:
plain0bits=convert_string_to_binary(plain0)
print plain0bits
print '******************************'
plain1bits=convert_string_to_binary(plain1)
print plain1bits

01100101001100100101101000110010001111000011100101010001010000100010100101100100001000010010101001010111010110100000000000000000
******************************
01100101001100100101101000110010001111000011100101010001010000100010100101100100001000010010101001010111010110100000000000000000


In [27]:
count=0
for i in plain0bits[::-1]:
    if i is '0':
        count+=1
    else:
        break
removezeros=count//8*8
print removezeros

16


In [28]:
mykey=plain0bits[:-16]
print mykey

0110010100110010010110100011001000111100001110010101000101000010001010010110010000100001001010100101011101011010


In [32]:
mykey_ASCII=convert_binary_to_hexbytes(mykey)
print mykey_ASCII

e2Z2<9QB)d!*WZ


In [30]:
# This will evaluate to false in my actual parameters
print mykey_ASCII == "lT4%&fFiP[fHSB"

False


# Exercise 4

In [27]:
###### Exercise 4
Q4_p = 259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077223
Q4_g = (259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 207254461881241936847382723924595103544871770663812380335177172367499249287309637473097159330731472813111313215131879600531552352195020851622948401875165687513934106264168097788619185022815140327306236090700132842333418922972894302738202671831113223899738729663831625542486140250723573146113060750970229236931)
Q4_h = (259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 29809145609893413370800351713524143884287988150162006681081775580489084279619129794788627738140514496435613987224720007723489304490257817240454128993828813595076868027736550133682501021994681293322208448053685963360625362419726178850190934694607764717408280652982233117448961333393123949765193856782412955947)
Q4_list = [((259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 205846156618700050624786347167403870058424025645169895972129845816372451368511568320055318117110853750865888668750159050679364255584315309144464315553839757511276217977968776762960509947929799064008065149242305611641169296534988996608607678007792721213312750866066423221902705862703094662836972338775902003515), 97319651529931006798475173432102036574364127612718236764482048602129125023488304953635967070977328929745191535038088690128062056434820152238473845431348540846186336537961743324963737824312900571382258139708821821274912690035075701390181468602181388408689873071851430430866604175119250555254235245619328800529), ((259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 219173918217651821428738224847103047575209778688459473233879814397631660227683558040060495832290731472513416619736620517091569298108509678503394653800552557990028526593164754060073378521682466855014010120909867500274461847064439303297805654848210187451291332953251988770354596843123826059911633009808050545551), 181614521958008203746980591555742490331711004914385852839566253728349928031034458947020320288647612654033937348513856708897924997359024151189424231495231416358682126284112527712693791055231536851167249512345456447860115428322952463579981025709445068637595851282395538666323890406282317423938459545298590659546), ((259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 41929916602706014511173383840122566770414052455409880533775726875081363298480602556662547903260731028234148493646771635791801403603903477087332168552198563746788886813720939513216079731857943818119105767853998646935918825880032216304373887201476976118260685208591532141783237133650067332738887116341070709455), 151224433313079993695094484364485935793615444473246229312755564653498681049349142455386482831938659017221145129259623705975280815724852592015659772453922243931096397158264250615692680784172427892694338964797462836503378928193507346190704838197893694692200844467886409527049475867966680020902534773459885959316), ((259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 172698438628752519310022392707026688967908913329493677922302282545444186060215630217026917983159757421805350149569591447814323079641284405106904385538364444510760928147269478090218158785438068160808173496286401019104059383931247210181229515977792345322166072207135539058710038534956795219121804117964123588596), 217843932170653821872577601829044911994263491532132077707653796615524080578860998609566195158854134345028697245896013552443409969795219269687661575103085963346342363506878852078699670305420777477161624461610325684675637175929907165125897468202118120515228601873234430516031942756508503636542436105364645420199), ((259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 31728268852828305940109090339286818225213729324584129917745140503092881976485640296746234516944889062233089523888441296659438670274311464901369284260529792595667581350689613949626492886145302547837726627499940801931677558099362109945625406062087439496128819291925972677805126198781791852363825658204740445161), 9377708092902287334225119289051586171599095440367472272479539100188541586561430748055896864011322310145450761592627840278255418920254399698866814681169804225345145333137821118157581133460620350611791298052513999805095026398811557349114944335148873309718305115319693367495012628470749135004238284076746856577), ((259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 180604508379512640007070007703382587514549225317806761713284089439604551879860237384984252497246861479401595517465983749280961578644754674416510159837963328124217245877905971651689498205313391469865801970913226570991324021426670810437595585732469165372007637361467111346122272891985190759036825941728142979119), 446995280197706690156988781186810850973885502328510022334345566027056900093715780550262144468419302993764986999109163885646117682705624106185117747006163563003859804266846261235042419795084154614753650611505652632663522113833758745737540867272254028230650620859165175749798082079113033742216797503409794735889), ((1, 191386815628975851210309679518913631765974145436440507392977515754863316654477766212829823394453084357216918648679476665976111772811046909811480284853482810809719189077596369101846428923072541848771486013768466629661715535009629272406692682523724207576790769353196962688907837000629977594993334187755689164574), 99569318483224827779728344196654202817209034617876965313777514637603183227309857977905770219758068559198205921668773791163266250858211346597770948672922668673754211116649492013951703937512307700460404512882914468294930101701267437669297931163874276461235100452951652130273790349279355599603066242850232754747), ((259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 33556152114358689522419313836331243907405203776374172889968194775900104663443428562957201796068079050320808699750818862457616087728934439978376591478665474091397974143768241008263438962547287977961767592291759656756892628493499711395981991319033567423667869370792827513066804325834020224840264754021222938019), 470354097443579054175169910457934977427908682926543204824759076528642512493961008010403489475155172019655794333527283382140732852512252489765526445706386761092465658443308795945994038699284550354512301634630177039033840978251827920839341626219649818805936480764839767148625510117944561420114445431418467903092), ((209579926214842048521438751093492250746655882720617635873402836685466375571204842402785563093651655752522171971769978847967597387360855940074077484458878191696630545265719816630548884754376895643716490170185868487913106289225115461125045348790898901759460078558032869722385675166438067053455556969156795221308, 163781364350647952033657820710955895041990415980125713340505061294209288393357016020073332007585308530845406090070117159862144520104477692654672058763382163265044242331465791611427023900669544946039010535465990400215362656714931925420263542290498186903861920238128886175609659690067730054101219032918078667426), 124740460248121083245629719434012625861876239087414845083722587080147521986836130554610917354260979976867835385315739625805528716980824495439274943470930323838153942184072821532589520355591420158235752662314348463711027651174471372726998228558896364469675884960967060580373130259002409219972164524869973690247), ((259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 212068031289949138982645113235746066343702948914955996704497680105545835883980627638082995566690239954361830719015389845177088264028188206977155935742277756470683030460511362601250900087321091720011872588587229395114725192715563675698855741893634136599686925346643031122487354351104752741713026319203062994508), 20383781840415558368222371781867593030667998890153065437322272426224395295254269020963977320764749583605894710201644026766620016058015399924462395135894100614298566404543499771404766909961935477035699439182983496171215834229420672718692881618271738516389883799484317174105506692903988805885262992199233550048), ((1, 23733301916476999228448791939892677059437589876457486589536040688297663667509598529736312924677740663893385432432852175726581663204780836488867610119768447178119197953059511059306762412884513099443047171166956019329900607584452904861987000474673373144408140359564626744259281351676244980686967377239348543062), 78244259949461703850618459800787363814082599322461698562893289097525895197529021632377308710992607950336230734771005178336009940441705880855724904067189507849512988212870380802256359998140552693821520206901127359881045935320894681488598676469315746075618362375719781269908703828894377297525807616037375012260), ((259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 75121156087590895335960355312592384733610117160765058430537645911157757017203506123819364980129734164036349474666266541080247852735388457939333661410810594611439672728877699174421340522818029037032482601652038986835785164168458153850988920312487267269463634958065836197891258680408113578562113403758605796760), 429211820985058872810045167452908198300415090022646343620202635760610838043255320136641413712577693480628672338740261341204966331645135508293777662014880980669550591198687937884952041063958946133104355163330965409005178782163020552375558648283988558896268399038999787699471027683396246128568722335008181591611), ((209923516499863361339385752022577080489490411892735363418232243317973228607741123807510450425250221843653562876733169674420709276585316695224755831354745633763973515224229961762093287431115636936297090771953007367982222786664507385016437314637856139999737545256952786143931414632903593139625669434078000923151, 118998599616766093004497040669933273777711498550114870038885299623006463621348186944129325761400261551818951231066498019876065687753779339854190212287820551861977375816713779298371415571630624995926909766199276587770771599411464485514860871432091575558661250550578189437705034930263010930783650949972452093831), 178924210249095321913192596420050616628653976415648863136244915684476350941352127132456341848390271487255322746711175505334295252362354345842472085778765635403146824374890461315221839272685981482831069069218246135939386638112979398143332492437345837326757507902568087374143199937561961188894814216889487267863), ((259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 209536662487725005825094458851954482135222390878054390071174604479137373472720293083272293800881981695681272833030765260723006935258170436679781115779975535358938031950353525711293270403801463336127107821357544754619462513993455800878434172403110721575369762258167763263035900947865771735547625722932323384321), 381853658260524268118532656404939423849956347451740483560181244158018461479737939028527242597261657241093879858784217424689771847386742047120763202371185359558821767303241911490396420990888041163492567543310346923413329377737580409225689028773432770135697663804603527532468807525432489641589654410799261407160), ((259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 215897035049071601767627621380718485399682632584301444223902161521864526053501116426934191469955058398999760289661231349507465508714041342720918065688914777262849303798576819377660704942222906308813874108830391105595242212727741619051287287757401260885158293447656860890278845221566590592423989349131935709395), 67516117505335753040754326781226633372251811975804842226646210855151767954589078749190396830072462919048665302477281245074667421710116506565604130833699337953212591062475674912092967393522804956270342478763395728466761947314551545663116769388479990512976875521258237599505389067126824900313137133327960485756), ((259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 193080072510298351596495632394644371253036815208281169933567174320272839420638021716332601574371906861560877658857530432657252092509521795710789730101876598080915565857427574796236430247330480345819338086335143481017599146880432195442384037112215016561284294847468869621813881469286963496617058135408662294915), 357231775448479329652350239971150798977437878141631285615785345300251693383981888687766143110402628550941665445667815368328757615430893390365960057301156268660362088196202814970977117698151281433848068684869267830270806396481117804590623267926812397767330587537151610972350386506210205510868615876730476290617), ((259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 192187742360960887552748475663138067038038700797686505751483736436857399859784749783326625986651874860155061002216660737742035097187101518121531548385700430103255695567446211828016463606314923174506432099909278050435044287488844654933524214757278011378316530498993067768073808978022295960390538557706343425531), 492623143760464226131513195985258302459147993128398354792400902481727826186701738939896913275338938260318994626950849904355032153632076929124028053770220867525600364776970539017932095863630014228401784927501043681092739969359344409766600317314291505698555529188227999857785864172508123942859417260099908404613), ((259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 91161657191258940863871769335218359226147559817238687817345254849581638764532738727591222964920939309670062462876179424129785818999391145693850666718995974926523995419688911620124101840150304812511880783320667560155254673855117174094820157463172468681102164699437814198676253314289400821633441678403943146189), 246052265133130129952313907559409065629891053910669724446709159205469437386464235721207863379021029490166215514305749475991431141508240968686159745431064125613026370094490314170347200017631093362230151883854995730938473765821501784131818612203096853005350474279257976403921866885215766242913157576271097778726), ((1, 243057111070130883847689650087589825556596532481960413622015578741072215448051988286627760042081184066052264079641584273960193715757218953215257508288831239161188195110502293553251678282622096356659625887198525578126499219320779977721154367645380420844610362100245950215712167757284644627778890895390782531830), 405540447263552983676835149290360491037100011703493012285566802771850652929174548044034267649756592901220587023659187338897847611829669294997909613558647076511213929147801065691803499532115688687598328136155064138582057050816710624554757183833882893593323263786041967180478685792995373988603912048786915531810), ((1, 244890869901157906214470745528377343966194222039779099586025655402670573477379061050654628986787576616743030040738394373669501518366223645374570826882442437718612468090334810183317168442489331537224760557722494174481053416700007100465605908709395742152655764332310900380910851012487968209758245376308930491316), 267305435022360280676390872016587191632957589035333209170646479637761231613283424095506776728251287922074774558371965981329421706836126935910624793136666531802511661456146688390905573758545057502325296846334458983064275703080527934780755292081767978656125645255493217500366555364296355996156337770090086943569), ((1, 192065362623870237584975166060532222832915216978090513905756848952184623781845109178869601434981655166733617134159571630906590123839816973999080422971344872569177567936261635082483230746293902295372873139317584902958023148050145677003839868943676109393185590320207258271865890500502186078724119205790200317649), 451740479879363975438198525670594355566507646214435703984521786889482520766418017307704806596910949620853432956087119031809970367433544079910273155486609453038747446730122723431582123746167138330058590298355084141643281448545468589792507336050295438281506018477890900442951589733297887374177531355235949518629), ((259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 195069952341081756513826002231344366153543405538752203410161279848819007636100453655433049091500365500332135836363983317234522315640910898774914362943282876965332874695349807954708369142180731639016614220232774941608608866519967831231007460735453266531081008232834320952856762836928521402796919147854628071834), 136936960138883122288212032876270799135462777583366123275760175892160730578268643817437598087006802882378776511509374517985824811303408625225877194251678492980103842933194932185741561819591772904422621338862950723344945705654402004307997173167036132072838817076014110593877388479000087561687061822697938461614), ((1, 252560794318716150631951598297223888593078617669965545267710505334606438832045740748400667101287724383707321664674735304042460343726018198632221276533397890239352925489077684390336583783671908569830496752145109399278367560342013148933493774552789427646467342078321230114564135755171777841397905428361611773514), 242536186432809347643222208115042397893151044997475217539054929403061552072870030096633567159236445008519472083941350701943639936552275052772583376812887867850398958137194705628339037246745490282254213220364005349639583388706485950337527708691253676462185441294126813972686014835514016495927621016507186495490), ((259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 61724277982964680851020380698141933623846594109611651880005680936207297091678021900227827371915792702766363193193777335383987020709968720103224166141126969881891828911565877436201255620519195348623392926488326652971396591005764067765138839159281449235221683547057255140989536511239322286254861002525355656585), 228214264256086810870496790321260962587934325165564914329662365089713536379465094868156375009643743543960925154341312838607174027235920065240789214227541089225856716769072942504169762062250927782123575463647650707050582881273397464362650679616349675751196960120596649424298253832471170218643400538141672558085), ((1, 111172438936173232755521485689632508737231384260780913384383761940493404448147330108149443723000545849915595344255375713036284583161874515089028226124674718998254844328082401906800763366814470143848586716064093181905027516694400953466979740871351682093145871809357604732347237967215106504772986548902431977543), 205193763744542785617942238211802599397262507814294804538102294127908047836627707865931670309995597960297622769096494537523167090414872104384315402669068023914019861831346769736183597539106988252781943452192641027065540886697734212885068716431385503784754204341820337551262443875276986184794625452380294739585), ((1, 233053630144518039476099560289228432011136113758443197307754320350058188464368581722323712183699123316152841179225804300412542409219081975871366842289080687787715075545746314308308433787700432400626247368797149893634521514431192772935032986360314081568546844847119250003334579031786737968755239101487652624034), 104853836743334642037967293362709525180103846333752709443930878343818121004066384074067173195560301880192797872523439970831274026660877701500455104586154347158940108979060145375745910729475681293681834043660798916520601787810657310940259864228942899945750870951726439890249275423564734565933901764246687304337), ((1, 38834489351253244778121269886268993890827982696673485448827750982056861494290677990360934711692540390913928604336968943289940379376554124229975424826675226627392045195627666098527949452557037154218342993444454594995764474873444851804790710274197033212624511940651728819118810240057713008419060193327156291940), 14871387364104775829577331748659137055470112515936871552958074510378833278231679577567350747923768928640719275180059699009027484618089727782588119842064600915462369625475013440872760425264701300183894616557095608482616169418166632734474883689121122683884693757904126042692882676011309304159448830613579491179), ((177414778433160947107525939710496146015601352778674319296940515108600073889641665511238935544318397151544794205743567597401682008402957186253493857915318639948203509841657289629342130022586554347369227829330867222706120354018676423314896207484451484677201679743043275866297149605389082238492109346703763416347, 5790677603468316236669495697917323390512988328532926698124965170925879188729335868134694846645713836017326520272109236572556842054073563555391961049530220677365924841060569632107943627531567286967688099431949625170232931008361379667658753809322945245902009822682770699887909101773159907802790062840433807352), 493041530207875593934943420932188122386813885624174774405354072091359166108634452628594981270174508193061060481863793102143329497550823567448436993835184327779469436372021518113934401328419111402656577483260169734282731971150259626365436776573453502163825217409305486257282735833378244551244024188709209949788)]
Q4_c = (259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 80050038767318093784936959028917382930500587644575465166391211580644680682865835458977288035453640942310103222055155439643499802637720517298581077005114994695049181385408287677778779152999786241827365503921553574471671321765164178228293616954108910606261354198604342922330116183340339003234044968459499793662)


In [28]:
print len(Q4_list)
print Q4_list[0][0] # This is the commitment
print Q4_list[0][1] # This is the message

28
(259291118756159055014506798395646936659406335338150373968565443660870131645084419976664428709201120592782597238611341578638911392362432564779152154081423725665932084547454627353142104991330562447317845082292140494747508440332163770083918368391107163832776753387655616448097153143502056381246339167353058077222, 205846156618700050624786347167403870058424025645169895972129845816372451368511568320055318117110853750865888668750159050679364255584315309144464315553839757511276217977968776762960509947929799064008065149242305611641169296534988996608607678007792721213312750866066423221902705862703094662836972338775902003515)
97319651529931006798475173432102036574364127612718236764482048602129125023488304953635967070977328929745191535038088690128062056434820152238473845431348540846186336537961743324963737824312900571382258139708821821274912690035075701390181468602181388408689873071851430430866604175119250555254235245619328800529


In [29]:
p=Q4_p
g=(mod(Q4_g[0],p),mod(Q4_g[1],p))
h=(mod(Q4_h[0],p),mod(Q4_h[1],p))
c=(mod(Q4_c[0],p),mod(Q4_c[1],p))

In [30]:
def group_operation((a,u),(b,v)):
    a=mod(a,p)
    u=mod(u,p)
    b=mod(b,p)
    v=mod(v,p)
    return (a*b,b*u+a*v)

def invert((a,u)):
    a=mod(a,p)
    u=mod(u,p)
    b=a^-1
    v=-b^2*u
    return (b,v)

def power((a,u),m):
    a=mod(a,p)
    u=mod(u,p)
    return (a^m,m*a^(m-1)*u)

def discrete_logarithm((c,d),(a,u)):
    """If the first argument is 8, and the second argument is 2, then the result is 3."""
    c=mod(c,p)
    d=mod(d,p)
    a=mod(a,p)
    u=mod(u,p)
    return d*c^-1*a*u^-1

In [31]:
# Can c_i be a commitment of m_i?
hiding=[]
for i in Q4_list:
    commitment=i[0]
    message=i[1]
    temp=power(invert(g),message)
    temp=group_operation(temp,commitment)
    r=discrete_logarithm(temp,h)
    if h[0]^r == temp[0]:
        hiding.append(1)
    else:
        hiding.append(0)
print hiding

[0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0]


In [32]:
# This will evaluate to false in my actual parameters
print hiding == [1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1]

False


In [33]:
# Part 2
m=100000
temp=power(g,m)
temp=invert(temp)
temp=group_operation(temp,c)
r=discrete_logarithm(temp,h)
print r

21437966882385609728785882962150168177187554462481270022546619837881942657930211890910429776428526327349464002510639328279769388671822670544616054890668107188929665499982606229573136350368877446233036388904269287828582331529695908328282521232190487630347448522390251716246502086308200313242258507648907847078


In [34]:
# This will evaluate to false in my actual parameters
print r == 14838106086731374594421304137342393835680213212344221452086405111055953176120714746826519631370149006540102636493996594733833005835129469335398740825096404590404219086044674080277213491659849979215504227597957410522415854061990294601765264630123192713779857566005956238665282102593135780092940464539682968716

False
